In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# code from https://towardsdatascience.com/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")
# type(dataset)

In [ ]:
import json
with open('/kaggle/input/mashqa-train/train_webmd_squad_v2_full.json', 'r') as file:
    d = json.load(file)

QUERY_SEPARATOR = "<QSEP>"
ANS_START = "<ANSSTART>"
ANS_END = "<ANSEND>"
EMPTY = "<EMPTY>"
data = d['data']
inputs = []
outputs = []
mx_len = 0
MAX_LEN = 2000
for dt in data:
    for paragraph in dt['paragraphs']:
        for qa in paragraph['qas']:
            assert(len(qa['answers']) == 1) # passed for train_full
            # input.append(qa['question'] + QUERY_SEPARATOR + paragraph['context'])
            inp = qa['question'] + QUERY_SEPARATOR
            ans = ""
            for idx, sent in enumerate(paragraph['sent_list']):
                inp += sent + " "
                if idx in qa['answers'][0]['answer_span']:
                    ans += sent + " "
                if len(inp) >= MAX_LEN:
                    inputs.append(inp)
                    if ans == "":
                      ans = EMPTY
                    outputs.append(ans)
                    mx_len = max(mx_len, len(outputs[-1]))
                    mx_len = max(mx_len, len(inputs[-1]))
                    inp = qa['question'] + QUERY_SEPARATOR
                    ans = ""
            if inp != qa['question'] + QUERY_SEPARATOR:
                inputs.append(inp)
                if ans == "":
                  ans = EMPTY
                outputs.append(ans)
                mx_len = max(mx_len, len(outputs[-1]))
                mx_len = max(mx_len, len(inputs[-1]))
                inp = qa['question'] + QUERY_SEPARATOR
                ans = ""
mx_len

In [ ]:
outputs[:10]

In [ ]:
# from transformers.data import Dataset
from datasets import Dataset

changed = True


In [ ]:
train_dataset = []
for x, y in zip(inputs, outputs):
    train_dataset.append("<s>[INST]<<SYS>><</SYS>>" +x+"[/INST]"+y+" </s>")

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
print(type(dataset))
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if changed:
    QUERY_SEPARATOR = "<QSEP>"
    ANS_START = "<ANSSTART>"
    ANS_END = "<ANSEND>"
    EMPTY = "<EMPTY>"
    tokenizer.add_tokens(QUERY_SEPARATOR)
    #   tokenizer.add_tokens(ANS_START)
    #   tokenizer.add_tokens(ANS_END)
    tokenizer.add_tokens(EMPTY)
#     # Tokenize the texts
#     tokenized_inputs = tokenizer(inputs, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

#     # Tokenize and split the outputs
#     tokenized_outputs = tokenizer(outputs, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
    #   dataset = train_dataset
    dataset = Dataset.from_dict({"text": train_dataset[:200]})


tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "<<SYS>><</SYS>> \n\n Gastritis is an inflammation, irritation, or erosion of the lining of the stomach. It can occur suddenly (acute) or gradually (chronic). Gastritis can be caused by irritation due to excessive alcohol use, chronic vomiting, stress, or the use of certain medications such as aspirin or other anti-inflammatory drugs. It may also be caused by any of the following: Helicobacter pylori ( H. pylori): A bacteria that lives in the mucous lining of the stomach; without treatment, the infection can lead to ulcers, and in some people, stomach cancer.  What are the symptoms of gastritis?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])